In [2]:
%%time
# Python 2.7

%matplotlib nbagg
%matplotlib inline 

import sys
sys.path
sys.path.append('..')
print(sys.version)

import os
import cPickle 
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import lasagne 
import lasagne.layers as L

from sklearn.utils import shuffle

import tools as tls


from data_loaders import svhn



2.7.12 |Continuum Analytics, Inc.| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


Using gpu device 0: GeForce GTX 950M (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5004)


Wall time: 28 s


In [6]:
### GLOBAL PARAMETERS ###

### META - HOW THE PROGRAM WORKS
file_name = 'data_c3'


### HYPER PARAMETERS
# eq_size = 1
# iw_size = 1
batch_size = 128
max_epoch = 100


learning_rate = 1e-2
L1 = 0
L2 = 0



C:\Users\tokee\Dropbox\SchoolWork\BSc5_DTU\DL\VAE\code\kingma\data\data_c3.pkl


In [10]:
%%time

### LOAD DATA
full_path = os.path.join(os.getcwd(), 'data')
full_path = os.path.join(full_path, file_name)
full_path  += '.pkl'
print(full_path)

import cPickle as pickle
with open(full_path, 'rb') as f:
    x_trai, t_trai, x_vali, t_vali, x_test, t_test = pickle.load(f)

print('')
print('Train shape: ', 
      x_trai.get_value().shape, t_trai.eval().shape)
print(type(x_trai))

print('Valid shape: ', 
      x_vali.get_value().shape, t_vali.eval().shape)
print(type(x_vali))

print('Test shape:  ', 
      x_test.get_value().shape, t_test.eval().shape)
print(type(x_test))
print('')

C:\Users\tokee\Dropbox\SchoolWork\BSc5_DTU\DL\VAE\code\kingma\data\data_c3.pkl

('Train shape: ', (32943L, 3072L), (32943L, 10L))
<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>
('Valid shape: ', (2258L, 3072L), (2258L, 10L))
<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>
('Test shape:  ', (12130L, 3072L), (12130L, 10L))
<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>

Wall time: 1.16 s
